In [ ]:
import os
import gc
import time
import collections
import json
import itertools
import functools
import datetime
import warnings

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import sklearn.model_selection
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.feature_selection
import sklearn.metrics
import sklearn.decomposition
import sklearn.manifold
import sklearn.linear_model
import sklearn.svm
import sklearn.ensemble
import sklearn.base
import umap

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import skorch
import skorch.scoring

In [ ]:
from pathwayae.models import MLP, Autoencoder, VAE, PAAE, PAVAE, PACAE, NopLayer
from pathwayae.skorch_utils import ScoredNeuralNetAutoencoder

from pathwayae.losses import AE_MSELoss, VAELoss, build_beta_schedule

from pathwayae.utils import from_log2pk, to_log2pk, fpkm_to_tpm, sample_wise_preprocess_fn, sigmoid, logcurve, logcurve_start_end

from pathwayae.pathway_utils import read_pathway_from_json_file

In [ ]:
data_folder = os.path.expanduser("~/data/") 
tcga_folder = os.path.join(data_folder, "pathwayae", "tcga")
meta_folder = os.path.join(data_folder, "pathwayae", "metabric")
os.makedirs(tcga_folder, exist_ok=True)

In [ ]:
cancer_type = "BRCA"

In [ ]:
metabric_ensembl_counts_gex_tsv_fname = os.path.join(meta_folder, f"data_mrna_agilent_microarray.txt.gz")
gex_meta = pd.read_csv(metabric_ensembl_counts_gex_tsv_fname, sep="\t", index_col="Hugo_Symbol").drop(columns="Entrez_Gene_Id")
gex_meta.index.rename("SampleID", inplace=True)
gex_meta = gex_meta.T.dropna(axis="columns")
gex_meta.columns = gex_meta.columns.str.upper()

gex_meta.shape

In [ ]:
gex_meta.index[:10]

In [ ]:
metabric_phenotype_csv_fname = os.path.join(meta_folder, f"data_clinical_patient.txt.gz")
metabric_phenotype = pd.read_csv(metabric_phenotype_csv_fname, sep="\t", comment="#", index_col="PATIENT_ID")
metabric_phenotype.index.rename("SampleID", inplace=True)
metabric_phenotype.rename(columns={"CLAUDIN_SUBTYPE": "PAM50"}, inplace=True)
metabric_phenotype = metabric_phenotype[["PAM50"]]
metabric_phenotype.dropna(inplace=True)
metabric_phenotype.drop(index=metabric_phenotype.index[metabric_phenotype["PAM50"] == "claudin-low"], inplace=True)
metabric_phenotype.drop(index=metabric_phenotype.index[metabric_phenotype["PAM50"] == "NC"], inplace=True)

for c in metabric_phenotype.columns:
    print(c, metabric_phenotype[c].count(), metabric_phenotype[c].value_counts())

metabric_phenotype.index[:10]

In [ ]:
assert sum(metabric_phenotype.PAM50.isna())==0

In [ ]:
metabric_phenotype["PAM50"].hist()

In [ ]:
ensembl_fpkm_gex_tsv_fname = os.path.join(tcga_folder, f"TCGA-{cancer_type}.htseq_fpkm.tsv.gz")
gex = pd.read_csv(ensembl_fpkm_gex_tsv_fname, sep="\t", index_col="Ensembl_ID").T.dropna(axis="columns")

zero_variance_columns = set(gex.var()[gex.var()==0].index)

gex = gex.drop(columns=list(zero_variance_columns))

with open(os.path.join(tcga_folder, "ensembl_to_gene_id.json")) as f:
    ensembl_to_gex_dict = json.load(f)

if True:
    columns_to_drop = [k for k in ensembl_to_gex_dict if ensembl_to_gex_dict[k]=="" and k not in zero_variance_columns]

    gex = gex.drop(columns=columns_to_drop)

    
gex.shape

In [ ]:
if True:
    value_counts = {}
    for k,v in ensembl_to_gex_dict.items():
        if v == '':
            continue
        elif k in gex.columns:
            if v in value_counts:
                value_counts[v].append(k)
            else:
                value_counts[v] = [k]
    value_counts = {k:v for k,v in value_counts.items() if len(v)>1}
    print(len(value_counts), sum((len(v) for v in value_counts.values())))

In [ ]:
duplicate_and_in_both = ["MATR3", "EMG1", "TMSB15B", "BMS1P4", "POLR2J4",]

for v in duplicate_and_in_both:
    display(gex[value_counts[v]].describe())
    display(gex[value_counts[v]].corr())
    display(gex_meta[v].describe())

[(v,value_counts[v]) for v in duplicate_and_in_both]

In [ ]:
for g_name, g_ensembl_ids in [(v,value_counts[v]) for v in duplicate_and_in_both]:
    n_ambiguous = len(g_ensembl_ids)
    fig, axes = plt.subplots(3+n_ambiguous,1, sharex=True, sharey=True, figsize=[plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1]*1.5])
    to_log2pk(from_log2pk(gex[g_ensembl_ids], 1).sum(axis=1), 1).hist(bins=20, ax=axes[n_ambiguous])
    axes[n_ambiguous].set_title(f"sum {g_name} (tcga, fpkm)")
    to_log2pk(from_log2pk(gex[g_ensembl_ids], 1).mean(axis=1), 1).hist(bins=20, ax=axes[n_ambiguous+1])
    axes[n_ambiguous+1].set_title(f"mean {g_name} (tcga, fpkm)")
    gex_meta[g_name].hist(bins=20, ax=axes[n_ambiguous+2])
    axes[n_ambiguous+2].set_title(f"{g_name} (metabric, counts)")
    for i, g_e_idx in enumerate(g_ensembl_ids):
        gex[g_e_idx].hist(bins=20, ax=axes[i])
        axes[i].set_title(f"{g_e_idx} (tcga, fpkm)")
    axes[n_ambiguous+2].set_xlabel("log2p1")
    fig.tight_layout()
    plt.show()
    plt.close()

In [ ]:
# Merge values that map to the same gene symbol
for g_name, g_ensembl_ids in value_counts.items():
    n_ambiguous = len(g_ensembl_ids)
    gex[g_name] = to_log2pk(from_log2pk(gex[g_ensembl_ids], 1).mean(axis=1), 1)
    gex.drop(columns=g_ensembl_ids, inplace=True)

In [ ]:
# Rename the rest
gex = gex.rename(columns=ensembl_to_gex_dict)

gex.columns.rename("GeneName", inplace=True)
gex.index.rename("SampleID", inplace=True)

In [ ]:
gex.shape, gex_meta.shape

In [ ]:
genes_in_both = sorted(set(gex_meta.columns).intersection(set(gex.columns)))
len(genes_in_both)

In [ ]:
gex = gex.loc[:,genes_in_both]
gex_meta = gex_meta.loc[:,genes_in_both]
gex.shape, gex_meta.shape

In [ ]:
assert all(map(lambda x: x[0]==x[1], list(zip(gex.columns, gex_meta.columns))))

In [ ]:
pathway_folder = os.path.join(data_folder, "pathways")
os.makedirs(pathway_folder, exist_ok=True)

In [ ]:
gex_genes = set(gex.columns.values)
gex_genes_indexer = {v:i for i,v in enumerate(gex.columns.values)}
get_pathways_with_indices = lambda pathways: [[gex_genes_indexer[gene] for gene in pathway] for pathway in pathways]

In [ ]:
kegg_pathways = read_pathway_from_json_file(os.path.join(pathway_folder,"c2.cp.kegg.v7.5.1.json"), gex_genes)
kegg_pathways_with_indices = get_pathways_with_indices(kegg_pathways)
number_of_pathways = len(kegg_pathways_with_indices)
pathways_input_dimension = sum((len(pathway) for pathway in kegg_pathways_with_indices))
number_of_input_genes = len(functools.reduce(lambda acc_p, p: acc_p.union(set(p)), kegg_pathways_with_indices, set()))
number_of_pathways, pathways_input_dimension, number_of_input_genes

In [ ]:
hallmark_pathways = read_pathway_from_json_file(os.path.join(pathway_folder,"h.all.v7.5.1.json"), gex_genes)
hallmark_pathways_with_indices = get_pathways_with_indices(hallmark_pathways)
number_of_pathways = len(hallmark_pathways_with_indices)
pathways_input_dimension = sum((len(pathway) for pathway in hallmark_pathways_with_indices))
number_of_input_genes = len(functools.reduce(lambda acc_p, p: acc_p.union(set(p)), hallmark_pathways_with_indices, set()))
number_of_pathways, pathways_input_dimension, number_of_input_genes

In [ ]:
ensembl_fpkm_phenotype_tsv_fname = os.path.join(tcga_folder, f"TCGA.{cancer_type}.sampleMap_{cancer_type}_clinicalMatrix")
phenotype = pd.read_csv(ensembl_fpkm_phenotype_tsv_fname, sep="\t", index_col="sampleID")
phenotype.index = phenotype.index.rename("SampleID")
phenotype = phenotype[[c for c in phenotype.columns if "pam50" in c.lower()]]
for c in phenotype.columns:
    print(phenotype[c].count(), phenotype[c].value_counts())

In [ ]:
phenotype_clf_tgt = "PAM50Call_RNAseq"
phenotype_clf_tgt_meta = "PAM50"
phenotype_clf_map = {
    "LumA":0,
    "LumB":1,
    "Basal":2,
    "Normal":3,
    "Her2":4,
}
phenotype_clf_nan = {f"{value}":np.nan for value in [np.nan, "not reported", ""]}
phenotype.columns

In [ ]:
# Drop nan
PHENOTYPE_CLF_COLUMN = "subtype"
phenotype[PHENOTYPE_CLF_COLUMN] = phenotype[phenotype_clf_tgt].replace(phenotype_clf_nan)
phenotype = phenotype.dropna(subset=[PHENOTYPE_CLF_COLUMN])
phenotype.columns

In [ ]:
_possible_mappings = {idx:[] for idx in phenotype.index}
for idx in phenotype.index:
    for v in gex[gex.index.str.startswith(idx)].index.values:
        _possible_mappings[idx].append(v)
_replacements = {k:sorted(v)[0] for k,v in _possible_mappings.items() if len(v)>0}
phenotype = phenotype.rename(index=_replacements, inplace=False)
phenotype.columns

In [ ]:
both_index = sorted(set(phenotype.index).intersection(gex.index))
[(len(idx), idx[:5],) for idx in [gex.index, phenotype.index, both_index]]

In [ ]:
both_index_meta = sorted(set(metabric_phenotype.index).intersection(gex_meta.index))
[(len(idx), idx[:5],) for idx in [gex_meta.index, metabric_phenotype.index, both_index_meta]]

In [ ]:
gex = gex.loc[both_index]
gex_meta = gex_meta.loc[both_index_meta]

full_phenotype = phenotype
phenotype_meta = full_phenotype_meta = metabric_phenotype

phenotype = phenotype.loc[both_index,[PHENOTYPE_CLF_COLUMN,]]
phenotype_meta = full_phenotype_meta.loc[both_index_meta,["PAM50"]]
phenotype[PHENOTYPE_CLF_COLUMN] = phenotype[PHENOTYPE_CLF_COLUMN].replace(phenotype_clf_map)
phenotype_meta[PHENOTYPE_CLF_COLUMN] = phenotype_meta["PAM50"].replace(phenotype_clf_map)
phenotype[phenotype_clf_tgt] = full_phenotype.loc[phenotype.index,phenotype_clf_tgt]
phenotype_meta[phenotype_clf_tgt_meta] = phenotype_meta.loc[phenotype_meta.index,phenotype_clf_tgt_meta]


(
    phenotype[PHENOTYPE_CLF_COLUMN].value_counts(), phenotype[PHENOTYPE_CLF_COLUMN].dtype, phenotype[PHENOTYPE_CLF_COLUMN].unique(), phenotype[PHENOTYPE_CLF_COLUMN].describe(),
    "\n",
    phenotype_meta[PHENOTYPE_CLF_COLUMN].value_counts(), phenotype_meta[PHENOTYPE_CLF_COLUMN].dtype, phenotype_meta[PHENOTYPE_CLF_COLUMN].unique(), phenotype_meta[PHENOTYPE_CLF_COLUMN].describe(),
)

In [ ]:
hue_order = sorted(phenotype[phenotype_clf_tgt].unique())
sns.displot(data=phenotype, y=phenotype_clf_tgt, hue=phenotype_clf_tgt, hue_order=hue_order)
plt.show()
sns.displot(data=phenotype_meta, y=phenotype_clf_tgt_meta, hue=phenotype_clf_tgt_meta, hue_order=hue_order)

In [ ]:
assert(all((gi==pi for gi,pi in zip(gex.index.to_list(), phenotype.index.to_list()))))

In [ ]:
assert(all((gi==pi for gi,pi in zip(gex_meta.index.to_list(), phenotype_meta.index.to_list()))))

In [ ]:
def plot_2d_space(df, SpaceTransformer=sklearn.decomposition.PCA, **kwargs):
    values = SpaceTransformer().fit_transform(df)
    return sns.scatterplot(x=values[:,0], y=values[:,1], **kwargs)

In [ ]:
plot_2d_space(gex, hue=phenotype[phenotype_clf_tgt])

In [ ]:
plot_2d_space(gex, SpaceTransformer=umap.UMAP, hue=phenotype[phenotype_clf_tgt])

In [ ]:
plot_2d_space(gex_meta, hue=phenotype_meta[phenotype_clf_tgt_meta])

In [ ]:
plot_2d_space(gex_meta, SpaceTransformer=umap.UMAP, hue=phenotype_meta[phenotype_clf_tgt_meta])

In [ ]:
genes_dim = gex.values.shape[1]
gex.values.shape[1]

In [ ]:
MAX_EPOCHS = 1024
BETA_START_AND_DURATION_LIST = [(32,128)]
DEFAULT_LR = 1e-4
BATCH_SIZE = int(gex.values.shape[0])
MAX_PATIENCE = 8
DEFAULT_EARLY_STOPPING_THRESHOLD = 0.001
EARLY_STOPPING_TEST_SIZE = 0.1
DEFAULT_SINK = lambda x:x
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

build_early_stopping = lambda: skorch.callbacks.EarlyStopping(
    patience = 16+MAX_PATIENCE,
    threshold = DEFAULT_EARLY_STOPPING_THRESHOLD,
    sink = DEFAULT_SINK
)


kegg_p = [torch.tensor(pathway) for pathway in kegg_pathways_with_indices]
hmrk_p = [torch.tensor(pathway) for pathway in hallmark_pathways_with_indices]

pways_keys_lst = ["KEGG", "Hallmark Genes",]
pways_defs_lst = [kegg_p, hmrk_p,]

paae_pipes = {
    **{
        f"PAAE-{pathway_hidden_dims}-{hidden_and_enc_dims} ({pways_key})": sklearn.pipeline.Pipeline(
            [
                ("scale", sklearn.preprocessing.QuantileTransformer(
                        n_quantiles=max(*gex.values.shape, *gex_meta.values.shape,),
                        output_distribution="normal",
                        )
                ),
                (
                    "net", 
                    ScoredNeuralNetAutoencoder(
                        PAAE,
                        module__genes_dim = genes_dim,
                        module__pathway_definitions = pways,
                        module__pathway_hidden_dims = pathway_hidden_dims,
                        module__hidden_dims = hidden_and_enc_dims[:-1],
                        module__encoding_dim = hidden_and_enc_dims[-1],
                        max_epochs = MAX_EPOCHS,
                        lr=DEFAULT_LR,
                        iterator_train__shuffle=True,
                        criterion = AE_MSELoss,
                        optimizer = torch.optim.Adam,
                        callbacks=[
                        ],
                        callbacks__print_log__sink = DEFAULT_SINK,
                        device=device,
                    )
                ),
            ],
        )
        for pways_key, pways in zip(
            pways_keys_lst,
            pways_defs_lst,
        )
        for hidden_and_enc_dims in ([64],[128,64])
        for pathway_hidden_dims in ([],[32],[32,16])
    },
}

ae_pipes = {
    **{
        f"AE-{hidden_and_enc_dims}": sklearn.pipeline.Pipeline(
            [
                ("scale", sklearn.preprocessing.QuantileTransformer(
                        n_quantiles=max(*gex.values.shape, *gex_meta.values.shape,),
                        output_distribution="normal",
                        )
                ),
                (
                    "net", 
                    ScoredNeuralNetAutoencoder(
                        Autoencoder,
                        module__input_dim = genes_dim,
                        module__hidden_dims = hidden_and_enc_dims[:-1],
                        module__encoding_dim = hidden_and_enc_dims[-1],
                        max_epochs = MAX_EPOCHS,
                        lr=DEFAULT_LR,
                        iterator_train__shuffle=True,
                        criterion = AE_MSELoss,
                        optimizer = torch.optim.Adam,
                        callbacks=[
                        ],
                        callbacks__print_log__sink = DEFAULT_SINK,
                        device=device,
                    )
                ),
            ],
        )
        for hidden_and_enc_dims in (
                [128,64], [256,128,64], [512,256,128,64],
                )
    }
}

vae_pipes = {
    **{
        f"VAE-{beta_schedule_type}-β{beta}-{hidden_and_enc_dims}": sklearn.pipeline.Pipeline(
            [
                ("scale", sklearn.preprocessing.QuantileTransformer(
                        n_quantiles=max(*gex.values.shape, *gex_meta.values.shape,),
                        output_distribution="normal",
                        )
                ),
                (
                    "net",
                    ScoredNeuralNetAutoencoder(
                        VAE,
                        module__input_dim = genes_dim,
                        module__hidden_dims = hidden_and_enc_dims[:-1],
                        module__encoding_dim = hidden_and_enc_dims[-1],
                        max_epochs = MAX_EPOCHS,
                        lr=DEFAULT_LR,
                        batch_size = BATCH_SIZE,
                        iterator_train__shuffle=True,
                        criterion = VAELoss,
                        criterion__original_loss = nn.MSELoss(),
                        criterion__beta_schedule = build_beta_schedule(beta_schedule_type, beta_start, beta_duration=beta_duration),
                        criterion__beta=beta,
                        criterion__kl_type="kingma",
                        optimizer = torch.optim.Adam,
                        callbacks=[
                        ],
                        callbacks__print_log__sink = DEFAULT_SINK,
                        device=device,
                    )
                ),
            ],
        )
        for beta in [1,5,10,50,100]
        for beta_start, beta_duration in BETA_START_AND_DURATION_LIST
        for beta_schedule_type in ("smooth", "step",)
        for hidden_and_enc_dims in ([128,64], [256,128,64], [512,256,128,64],)
    },
}

pavae_pipes = {
    **{
        f"PAVAE-{beta_schedule_type}-β{beta}-{pathway_hidden_dims}-{hidden_and_enc_dims} ({pways_key})": sklearn.pipeline.Pipeline(
            [
                ("scale", sklearn.preprocessing.QuantileTransformer(
                        n_quantiles=max(*gex.values.shape, *gex_meta.values.shape,),
                        output_distribution="normal",
                        )
                ),
                (
                    "net",
                    ScoredNeuralNetAutoencoder(
                        PAVAE,
                        module__genes_dim = genes_dim,
                        module__pathway_definitions = pways,
                        module__pathway_hidden_dims = pathway_hidden_dims,
                        module__hidden_dims = hidden_and_enc_dims[:-1],
                        module__encoding_dim = hidden_and_enc_dims[-1],
                        max_epochs = MAX_EPOCHS,
                        lr=DEFAULT_LR,
                        batch_size = BATCH_SIZE,
                        iterator_train__shuffle=True,
                        criterion = VAELoss,
                        criterion__original_loss = nn.MSELoss(),
                        criterion__beta_schedule = build_beta_schedule(beta_schedule_type, beta_start, beta_duration=beta_duration),
                        criterion__beta=beta,
                        criterion__kl_type="kingma",
                        optimizer = torch.optim.Adam,
                        callbacks=[
                        ],
                        callbacks__print_log__sink = DEFAULT_SINK,
                        device=device,
                    )
                ),
            ],
        )
        for beta in [1,5,10,50,100]
        for beta_start, beta_duration in BETA_START_AND_DURATION_LIST
        for beta_schedule_type in ("smooth", "step")
        for pways_key, pways in zip(
            pways_keys_lst,
            pways_defs_lst,
        )
        for hidden_and_enc_dims in ([128,64], [256,128,64],)
        for pathway_hidden_dims in ([],[32],[32,16])
    },
}

scale_pipe = sklearn.pipeline.Pipeline(
    [
        ("scale", sklearn.preprocessing.QuantileTransformer(
                n_quantiles=max(*gex.values.shape, *gex_meta.values.shape,),
                output_distribution="normal",
                )
        ),
    ],
)

ExperimentDefinition = collections.namedtuple("ModelDefinition", ["Model", "params"])

params = {
    "net__lr": [DEFAULT_LR],
    "net__max_epochs":  [MAX_EPOCHS],
}

models_to_test_dict = {
    "ae": {**{k: ExperimentDefinition(ae_pipes[k], params) for k in ae_pipes}},
    "z-norm": {"z-norm": ExperimentDefinition(scale_pipe, {})},
    "paae": {**{k: ExperimentDefinition(paae_pipes[k], params) for k in paae_pipes}},
    "vae": {**{k: ExperimentDefinition(vae_pipes[k], params) for k in vae_pipes}},
    "pavae": {**{k: ExperimentDefinition(pavae_pipes[k], params) for k in pavae_pipes}},
}


models_to_test = {
    **{k: ExperimentDefinition(ae_pipes[k], params) for k in ae_pipes},
    "z-norm": ExperimentDefinition(scale_pipe, {}),
    **{k: ExperimentDefinition(paae_pipes[k], params) for k in paae_pipes},
    **{k: ExperimentDefinition(vae_pipes[k], params) for k in vae_pipes},
    **{k: ExperimentDefinition(pavae_pipes[k], params) for k in pavae_pipes},
}

In [ ]:
results_folder = "results/metabric"
os.makedirs(results_folder, exist_ok=True)
images_folder = "images/metabric"
os.makedirs(images_folder, exist_ok=True)
SAVING_FORMATS = ["png", "pdf", "svg"]
for fmt in SAVING_FORMATS: os.makedirs(os.path.join(images_folder,fmt), exist_ok=True)

In [ ]:
DISABLE_TQDM = False

In [ ]:
paper_results_set = {
    "z-norm": "z-norm",
    "AE-[128, 64]": "AE-[128, 64]",
    "PAAE-[32]-[64] (KEGG)": "PAAE-[32]-[64] (KEGG)",
    "PAAE-[32]-[64] (Hallmark Genes)": "PAAE-[32]-[64] (Hallmark Genes)",
    "PAVAE-smooth-β1-[]-[128, 64] (KEGG)": "PAVAE-SVM-smooth-β1-[]-[128, 64] (KEGG)",
    "PAVAE-step-[]-[128,64] (Hallmark Genes)": "PAVAE-step-[]-[128,64] (Hallmark Genes)",
    "VAE-step-β1-[128, 64]": "VAE-step-β1-[128, 64]",

}
paper_normtype = "log2p1e-3_fpkm"

In [ ]:
paper_models_to_test = {k:v for k,v in models_to_test.items()}
paper_models_to_test = {paper_results_set[k]:v for k,v in paper_models_to_test.items() if k in paper_results_set}
normalization_types_to_test = [paper_normtype]
print(normalization_types_to_test)
len(paper_models_to_test), paper_models_to_test.keys()

In [ ]:
label_encoder = sklearn.preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(phenotype[PHENOTYPE_CLF_COLUMN])
y_test = label_encoder.transform(phenotype_meta[PHENOTYPE_CLF_COLUMN])

y = y_train
metrics = {
    "Accuracy": sklearn.metrics.make_scorer(sklearn.metrics.accuracy_score,),
    "Precision": sklearn.metrics.make_scorer(functools.partial(sklearn.metrics.precision_score, average="macro" if len(set(y))>2 else "binary",),),
    "Recall": sklearn.metrics.make_scorer(functools.partial(sklearn.metrics.recall_score, average="macro" if len(set(y))>2 else "binary",),),
    "F1": sklearn.metrics.make_scorer(functools.partial(sklearn.metrics.f1_score, average="macro" if len(set(y))>2 else "binary"),),
    "AUC": sklearn.metrics.make_scorer(functools.partial(sklearn.metrics.roc_auc_score, average="macro" if len(set(y))>2 else None, multi_class="ovr",), needs_proba=True),
}

CLASSIFIERS_TO_TEST = [sklearn.svm.SVC(probability=True), sklearn.linear_model.LogisticRegression(), sklearn.ensemble.RandomForestClassifier()]

In [ ]:
results_time = datetime.datetime.now()

models = {}
models_latent_spaces_train = {}
models_latent_spaces_test = {}
models_pathway_spaces_train = {}
models_pathway_spaces_test = {}
models_fit_time = {}
models_scores_train = {}
models_scores_test = {}

try:
    X_train = gex.values
    X_test = gex_meta.values

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    if not np.isfinite(X_train).all():
        raise ValueError(f"Some inputs were not finite")
    for model_name in tqdm(paper_models_to_test, desc="Model", leave = False, disable = DISABLE_TQDM,):
        Model:sklearn.pipeline.Pipeline = paper_models_to_test[model_name].Model
        model_fit_start = time.time()
        model = Model.fit(X_train)
        models_fit_time[model_name] = time.time() - model_fit_start
        if hasattr(model, "score"):
            models_scores_train[model_name] = model.score(X_train)
            models_scores_test[model_name] = model.score(X_test)
        
        z_train = X_train
        z_test = X_test
        print(model_name, models_fit_time[model_name])
        print("X", z_train.shape, z_test.shape)
        for step_name, step in model.steps:
            print(step_name, end=" ")
            if isinstance(step,ScoredNeuralNetAutoencoder):
                module_param = next(step.module_.parameters())

                # Get PAs first
                if "PA" in model_name:
                    with torch.no_grad():
                        p_train = step.module_.get_pathway_activities(torch.tensor(z_train, device=module_param.device, dtype=module_param.dtype))
                        p_test = step.module_.get_pathway_activities(torch.tensor(z_test, device=module_param.device, dtype=module_param.dtype))
                    
                    if isinstance(p_train, tuple):
                        p_train = p_train[0]
                    if isinstance(p_test, tuple):
                        p_test = p_test[0]

                    p_train = p_train.detach().cpu().numpy()
                    p_test = p_test.detach().cpu().numpy()
                    print("PA", p_train.shape, p_test.shape)
                
                # Then update Zs
                with torch.no_grad():
                    z_train = step.module_.encode(torch.tensor(z_train, device=module_param.device, dtype=module_param.dtype))
                    z_test = step.module_.encode(torch.tensor(z_test, device=module_param.device, dtype=module_param.dtype))

                if isinstance(z_train, tuple):
                    z_train = z_train[0]
                if isinstance(z_test, tuple):
                    z_test = z_test[0]

                z_train = z_train.detach().cpu().numpy()
                z_test = z_test.detach().cpu().numpy()

                print(z_train.shape, z_test.shape)
                break
            z_train = step.transform(z_train)
            if isinstance(step,sklearn.preprocessing.QuantileTransformer):
                z_test = sklearn.preprocessing.quantile_transform(z_test, n_quantiles=step.n_quantiles, output_distribution=step.output_distribution)
            else:
                z_test = step.transform(z_test)
            print(z_train.shape, z_test.shape)
        models[model_name] = model
        models_latent_spaces_train[model_name] = z_train
        models_latent_spaces_test[model_name] = z_test
        if "PA" in model_name:
            models_pathway_spaces_train[model_name] = p_train
            models_pathway_spaces_test[model_name] = p_test
except KeyboardInterrupt:
    pass
models_fit_time

In [ ]:
torch.cuda.empty_cache()
with torch.no_grad():
    torch.cuda.empty_cache()
gc.collect()

In [ ]:
class_ordering = sorted(phenotype_meta[phenotype_clf_tgt_meta].unique())
class_ordering

In [ ]:
models_latent_spaces_train.keys()

In [ ]:
model_spaces_test = {
    **{f"Input Space": (space, phenotype_meta[phenotype_clf_tgt_meta], "Metabric") for model, space in models_latent_spaces_test.items() if model.lower()=="z-norm"},
    **{f"{model} - Latent Space": (space, phenotype_meta[phenotype_clf_tgt_meta], "Metabric") for model, space in models_latent_spaces_test.items() if model.lower()!="z-norm"},
    **{f"{model} - Pathway Space": (space, phenotype_meta[phenotype_clf_tgt_meta], "Metabric") for model, space in models_pathway_spaces_test.items()},
}
model_spaces_train = {
    **{f"Input Space": (space, phenotype[phenotype_clf_tgt], "TCGA") for model, space in models_latent_spaces_train.items() if model.lower()=="z-norm"},
    **{f"{model} - Latent Space": (space, phenotype[phenotype_clf_tgt], "TCGA") for model, space in models_latent_spaces_train.items() if model.lower()!="z-norm"},
    **{f"{model} - Pathway Space": (space, phenotype[phenotype_clf_tgt], "TCGA") for model, space in models_pathway_spaces_train.items()},
}

In [ ]:
clustering_scores = {
    (model_name, space_name): {
        "silhouette_score": sklearn.metrics.silhouette_score(z, y),
        "davis_bouldin_index": sklearn.metrics.davies_bouldin_score(z, y),
        "calinski_harabasz_index": sklearn.metrics.calinski_harabasz_score(z, y)
    } for model_name, (z, y, space_name) in itertools.chain(model_spaces_train.items(), model_spaces_test.items())
}


clustering_scores_dict = {
    "Model": [],
    "Dataset": [],
    "Silhouette Score": [],
    "Calinski-Harabasz Index": [],
    "Davis-Bouldin Index": [],
}

for (model_name, space_name), CSs in clustering_scores.items():
    silhouette_score = clustering_scores[(model_name, space_name)]["silhouette_score"]
    calinski_harabasz_index = clustering_scores[(model_name, space_name)]["calinski_harabasz_index"]
    davis_bouldin_index = clustering_scores[(model_name, space_name)]["davis_bouldin_index"]
    clustering_scores_dict["Model"].append(model_name)
    clustering_scores_dict["Dataset"].append(space_name)
    clustering_scores_dict["Silhouette Score"].append(silhouette_score)
    clustering_scores_dict["Calinski-Harabasz Index"].append(calinski_harabasz_index)
    clustering_scores_dict["Davis-Bouldin Index"].append(davis_bouldin_index)

clustering_scores_df = pd.DataFrame(clustering_scores_dict)
clustering_scores_df.to_csv(os.path.join(results_folder,"clustering_scores.csv"))
clustering_scores_df

In [ ]:
for score in ["Silhouette Score", "Calinski-Harabasz Index", "Davis-Bouldin Index",]:
    sns.barplot(data=clustering_scores_df, x=score, y="Model", hue="Dataset")
    plt.show()
    plt.close()

In [ ]:
normtypes_to_show = None
clusion_names_to_show = None
plot_legend = False
savefigs = True
figsize_to_show = None#"wide"

for space_transformer in [umap.UMAP, sklearn.decomposition.PCA, sklearn.manifold.TSNE]:
    for model_name, (z, y, space_name) in itertools.chain(model_spaces_train.items(), model_spaces_test.items()):
        silhouette_score = clustering_scores[(model_name, space_name)]["silhouette_score"]
        calinski_harabasz_index = clustering_scores[(model_name, space_name)]["calinski_harabasz_index"]
        davis_bouldin_index = clustering_scores[(model_name, space_name)]["davis_bouldin_index"]
        pct_na = np.mean(np.isnan(z))
        if pct_na>0:
            continue
        for figsizename, figsize in [
            ("default",None),
            ("thin",(3,5)),
            ("wide",(7,5)),
        ]:
            plt.figure(figsize=figsize)
            plot_2d_space(z, SpaceTransformer=umap.UMAP, hue=y, hue_order=class_ordering, alpha=0.5,)
            plt.xticks([])
            plt.yticks([])
            plt.xlabel(f"{space_transformer.__name__} 1")
            plt.ylabel(f"{space_transformer.__name__} 2")
            plt.title(f"{model_name} (SS={silhouette_score:.3f}, CHI={calinski_harabasz_index:.3f}, DBI={davis_bouldin_index:.3f})")
            
            if not plot_legend:
                plt.legend([],[], frameon=False)
            if savefigs:
                model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')
                for fmt in SAVING_FORMATS: plt.savefig(os.path.join(images_folder,fmt,f"brca-scatter-{figsizename}-{space_transformer.__name__}-{space_name.lower()}-{model_fname}.{fmt}"), bbox_inches="tight")
            if figsize_to_show==figsizename and space_transformer.__name__=="UMAP":
                    print(model_name, space_name, figsizename)
                    plt.show()
            plt.close()

In [ ]:
hallmark_pathway_description_path = os.path.join(pathway_folder,"h.all.v7.5.1.json")

with open(hallmark_pathway_description_path, "r") as f:
    hallmark_pathway_descriptions = json.load(f)
hallmark_pathway_genes = [(k,hallmark_pathway_descriptions[k]["geneSymbols"]) for k in hallmark_pathway_descriptions]
hallmark_all_pathway_genes = functools.reduce(lambda acc, v: acc.union(set(v[1])), hallmark_pathway_genes, set())
hallmark_common_genes = hallmark_all_pathway_genes.intersection(gex_genes)
hallmark_pathway_genes_with_allowed_genes = [(k,[gene for gene in pathway if gene in hallmark_common_genes]) for k, pathway in hallmark_pathway_genes]

hallmark_pway_names = [k for (k,p), pi in zip(hallmark_pathway_genes_with_allowed_genes,hallmark_pathways_with_indices)]
assert(all([(len(p)==len(pi)) for (k,p), pi in zip(hallmark_pathway_genes_with_allowed_genes,hallmark_pathways_with_indices)]))
hallmark_pway_names = [pname.replace("HALLMARK_","") for pname in hallmark_pway_names]
len(hallmark_pway_names), hallmark_pway_names[:5]

In [ ]:
kegg_pathway_description_path = os.path.join(pathway_folder,"c2.cp.kegg.v7.5.1.json")

with open(kegg_pathway_description_path, "r") as f:
    kegg_pathway_descriptions = json.load(f)
kegg_pathway_genes = [(k,kegg_pathway_descriptions[k]["geneSymbols"]) for k in kegg_pathway_descriptions]
kegg_all_pathway_genes = functools.reduce(lambda acc, v: acc.union(set(v[1])), kegg_pathway_genes, set())
kegg_common_genes = kegg_all_pathway_genes.intersection(gex_genes)
kegg_pathway_genes_with_allowed_genes = [(k,[gene for gene in pathway if gene in kegg_common_genes]) for k, pathway in kegg_pathway_genes]

kegg_pway_names = [k for (k,p), pi in zip(kegg_pathway_genes_with_allowed_genes,kegg_pathways_with_indices)]
assert(all([(len(p)==len(pi)) for (k,p), pi in zip(kegg_pathway_genes_with_allowed_genes,kegg_pathways_with_indices)]))
kegg_pway_names = [pname.replace("KEGG_","") for pname in kegg_pway_names]
len(kegg_pway_names), kegg_pway_names[:5]

In [ ]:
pathway_set_to_use = "Hallmark Genes"
base_model_name_to_use = "PAAE-[32]-[64]"

pathway_set_to_use_dict = {
    "Hallmark Genes": hallmark_pway_names,
    "KEGG": kegg_pway_names,
}

clustermap_k_map = {
    "KEGG": 32,
    "Hallmark Genes": 50
}

featuremap_k_map = {
    "KEGG": 32,
    "Hallmark Genes": 50
}

violinplot_k_map = {
    "KEGG": 32,
    "Hallmark Genes": 50
}

pway_names = pathway_set_to_use_dict[pathway_set_to_use]
base_model_name = f"{base_model_name_to_use} ({pathway_set_to_use})"
base_model_name

In [ ]:
models_pathway_spaces_train[base_model_name].shape

In [ ]:
use_train = True
if use_train:
    pathway_space_to_use_dataset_name = "TCGA"
    pathway_space_to_use = pd.DataFrame(models_pathway_spaces_train[base_model_name], index=gex.index, columns=pway_names)
    phenotype_series_col = "PAM50Call_RNAseq"
    phenotype_to_use = phenotype
else:
    pathway_space_to_use_dataset_name = "Metabric"
    pathway_space_to_use = pd.DataFrame(models_pathway_spaces_test[base_model_name], index=gex_meta.index, columns=pway_names)
    phenotype_series_col = "PAM50"
    phenotype_to_use = phenotype_meta
    

In [ ]:
pway_df = pathway_space_to_use.copy()
pway_scores = sorted(zip(pway_df.columns,sklearn.feature_selection.mutual_info_classif(pway_df,phenotype_to_use[phenotype_series_col])), key=lambda x:x[1], reverse=True,)

model_name = f"{base_model_name} - Pathway Space"
model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')

pd.DataFrame(
    data = {"Pathway": [p for p,_ in pway_scores], "Mutual Information": [mi for _,mi in pway_scores]}
).set_index(
    "Pathway"
).to_csv(
    os.path.join(results_folder,f"brca-pathway-{pathway_space_to_use_dataset_name}-{model_fname}-mi-{results_time:%Y%m%d%H%M}.csv")
)

In [ ]:
show_k_best_pathways_k = 32
pway_df.columns[:show_k_best_pathways_k]

In [ ]:
replacements = []

s_to_e = {s:e for s,e in replacements}
e_to_s = {e:s for s,e in replacements}
for s, e in replacements:
    if (s_to_e[s]!=e):
        print(f"Start {s} has duplicate with end {e}")
    if (e_to_s[e]!=s):
        print(f"End {e} has duplicate with start {s}")

In [ ]:
MAX_PWAY_NAME_SIZE = 16

In [ ]:
show_k_best_pathways_k = clustermap_k_map[pathway_set_to_use]
plot_df = pway_df[[p for p,s in pway_scores[:show_k_best_pathways_k]]]
plot_df_cols = functools.reduce(lambda cols, mapping: cols.str.replace(*mapping,), replacements, plot_df.columns)
model_name = f"{base_model_name} - Pathway Space"

# Create a categorical palette to identify the networks
unique_phenotypes = phenotype[phenotype_clf_tgt].unique()

phenotype_pal = sns.color_palette()
phenotype_lut = dict(zip(unique_phenotypes, phenotype_pal))
phenotype_colors = pd.Series(phenotype_to_use[phenotype_series_col]).map(phenotype_lut)

for metric in ['correlation', 'cosine', 'euclidean',]:
    print(metric)
    try:
        try:
            g = sns.clustermap(plot_df.T, cmap="vlag",
                metric=metric,
                col_colors=phenotype_colors,
                col_cluster=True,
                row_cluster=True,
            )
        except KeyboardInterrupt:
            raise
        except:
            continue
        old_yticks = g.ax_heatmap.get_yticks()
        old_yticklabels = g.ax_heatmap.get_yticklabels()
        new_yticks = np.arange(min(old_yticks),min(old_yticks)+len(plot_df.columns),1)
        g.ax_heatmap.set_yticks(new_yticks, labels = plot_df_cols[g.dendrogram_row.reordered_ind].str[:MAX_PWAY_NAME_SIZE])
        g.ax_heatmap.set_xticklabels([])

        g.ax_col_dendrogram.remove()
        g.ax_cbar.remove()
        plt.title("")
        model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')
        for fmt in SAVING_FORMATS: g.savefig(os.path.join(images_folder,fmt,f"clustermap-BRCA-{pathway_space_to_use_dataset_name}-{model_fname}-{metric}.{fmt}"), dpi=600)
        plt.close()
    except KeyboardInterrupt:
        break


In [ ]:
phenotype_pal

In [ ]:
phenotype_lut

In [ ]:
show_k_best_pathways_k = 32
plot_df = pway_df[[p for p,s in pway_scores[:show_k_best_pathways_k]]]

# Create a categorical palette to identify the networks
unique_phenotypes = phenotype[phenotype_clf_tgt].unique()
phenotype_colors = pd.Series(phenotype_to_use[phenotype_series_col]).map(phenotype_lut)

g = sns.clustermap(plot_df[[p for p,s in pway_scores[:show_k_best_pathways_k]]].T,
                   col_colors=phenotype_colors,
                   col_cluster=True,
                   row_cluster=True,
)
old_yticks = g.ax_heatmap.get_yticks()
new_yticks = np.arange(min(old_yticks),min(old_yticks)+len(plot_df.columns),1)
g.ax_heatmap.set_yticks(new_yticks, labels = plot_df.columns[g.dendrogram_row.reordered_ind])
g.ax_heatmap.set_xticklabels([])
g.ax_col_dendrogram.remove()
g.ax_cbar.remove()

# Draw the full plot

In [ ]:
extra_pway_df = pway_df.copy()
values_umap = umap.UMAP().fit_transform(extra_pway_df.values)
for i in range(values_umap.shape[1]):
    extra_pway_df[f"UMAP {i}"] = values_umap[:,i]
model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')
extra_pway_df.to_csv(f"pway_and_umap_{model_fname}.csv")
phenotype.to_csv(f"pheno_{model_fname}.csv")

In [ ]:
extra_pway_df.columns

In [ ]:
show_k_best_pathways_k = 5
save_k_best_pathways_k = featuremap_k_map[pathway_set_to_use]

figsizename,figsize=("wide",(7,5))
plot_legend=True
savefigs=True
showfigs=[phenotype_series_col,] + [p for p,s in pway_scores[:show_k_best_pathways_k]]
model_name = f"{base_model_name} - Pathway Space"
model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')

col_score_dict = dict(pway_scores)
col_score_dict[phenotype_series_col] = sklearn.feature_selection.mutual_info_classif(phenotype_to_use[["subtype"]],phenotype_to_use[phenotype_series_col], discrete_features=True)[0]

for col in [phenotype_series_col] + [p for p,s in pway_scores[:save_k_best_pathways_k]]:
    if col in ["UMAP 0","UMAP 1"]: continue
    plt.figure(figsize=figsize)

    if col == phenotype_series_col:
        sns.scatterplot(data=extra_pway_df,x="UMAP 0",y="UMAP 1", hue=phenotype_to_use[phenotype_series_col], alpha=0.5,)
    else:
        sns.scatterplot(data=extra_pway_df,x="UMAP 0",y="UMAP 1", hue=col, palette="vlag", alpha=0.5)
    plt.title(f"{model_name} [MI:{col_score_dict[col]:.4f}]")
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(f"UMAP 0")
    plt.ylabel(f"UMAP 1")
    if not plot_legend:
        print(plt.legend())
        plt.legend([],[], frameon=False)
    if savefigs:
        
        for fmt in SAVING_FORMATS: plt.savefig(os.path.join(images_folder,fmt,f"brca-{pathway_space_to_use_dataset_name}-scatter-{model_fname}-UMAP-{figsizename}-{col}.{fmt}"), bbox_inches="tight")
    if col in showfigs:
        plt.show()
    plt.close()

In [ ]:
show_k_best_pathways_k = 5
save_k_best_pathways_k = violinplot_k_map[pathway_set_to_use]

figsizename,figsize=("wide",(7,5))
plot_legend=True
savefigs=True
showfigs=[phenotype_series_col,] + [ps[0] for ps in pway_scores[:show_k_best_pathways_k]]
model_name = f"{base_model_name} - Pathway Space"

col_score_dict = dict(pway_scores)
col_score_dict[phenotype_series_col] = sklearn.feature_selection.mutual_info_classif(phenotype_to_use[["subtype"]],phenotype_to_use[phenotype_series_col], discrete_features=True)[0]

for col in [p for p,s in pway_scores[:save_k_best_pathways_k]]:
    if col in ["UMAP 0","UMAP 1"]: continue
    plt.figure(figsize=figsize)
    sns.violinplot(data=extra_pway_df, y=col, x=phenotype_to_use[phenotype_series_col], hue=phenotype_to_use[phenotype_series_col], dodge=False)
    plt.title(f"{model_name} [MI:{col_score_dict[col]:.4f}]")
    if not plot_legend:
        print(plt.legend())
        plt.legend([],[], frameon=False)
    if savefigs:
        model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')
        for fmt in SAVING_FORMATS: plt.savefig(os.path.join(images_folder,fmt,f"brca-{pathway_space_to_use_dataset_name}-violin-{model_fname}-UMAP-{figsizename}-{col}.{fmt}"), bbox_inches="tight")
    if col in showfigs:
        plt.show()
    plt.close()

In [ ]:
model_name = base_model_name
model_fname = model_name.replace('(','_').replace(')','_').replace('-','_').replace(' ','')
torch.save(models[model_name]["net"].module_.state_dict(), os.path.join(results_folder,f"{model_fname}.pt"))

In [ ]:
torch.load(os.path.join(results_folder,f"{model_fname}.pt"))

In [ ]:
0